<a href="https://www.kaggle.com/code/gpreda/simple-sequential-chain-with-llama-2-and-langchain?scriptVersionId=144684662" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction


## Objective  

Use Llama 2 and Langchain to create a multi-step task chain. 

## Model details  

* **Model**: Llama 2  
* **Variation**: 7b-chat-hf    
* **Version**: V1  
* **Framework**: PyTorch  

LlaMA 2 model is pretrained and fine-tuned with 2 Trillion tokens and 7 to 70 Billion parameters which makes it one of the powerful open source models. It is a highly improvement over LlaMA 1 model.

# InstalIing, imports, utils

Install packages.

In [ ]:
!pip install transformers accelerate einops langchain xformers bitsandbytes chromadb sentence_transformers

Import packages.

In [ ]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time

from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain import PromptTemplate


## Initialize model, tokenizer, query pipeline  

Define the model, the device, and the bitsandbytes configuration.

In [ ]:
model_id = '/kaggle/input/llama-2/pytorch/7b-chat-hf/1'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

Prepare the model and the tokenizer.

In [ ]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

Define a pipeline. 

In [ ]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

llm = HuggingFacePipeline(pipeline=query_pipeline)



We test it by running a simple query.

In [ ]:
# checking again that everything is working fine
llm(prompt="What is the most popular city in France for tourists? Just return the name of the city.")

# Define and execute the sequential chain


We define a chain with two tasks in sequence.  
The input for the second step is the output of the first step.  

In [ ]:
def sequential_chain(country):
    """
    Args:
        country: country selected
    Returns:
        None
    """
    time_1 = time()
    template = "What is the most popular city in {country} for tourists? Just return the name of the city."

    #  first task in chain
    first_prompt = PromptTemplate(

    input_variables=["country"],

    template=template)

    chain_one = LLMChain(llm = llm, prompt = first_prompt)

    # second step in chain
    second_prompt = PromptTemplate(

    input_variables=["city"],

    template="What are the top three things to do in this: {city} for tourists. Just return the answer as three bullet points.",)

    chain_two = LLMChain(llm=llm, prompt=second_prompt)

    # combine the two steps and run the chain sequence
    overall_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)
    overall_chain.run(country)
    time_2 = time()
    print(f"Run sequential chain: {round(time_2-time_1, 3)} sec.")

In [ ]:
final_answer = sequential_chain("France")

In [ ]:
final_answer = sequential_chain("Germany")